In [1]:
import pandas as pd
import numpy as np
np.random.seed(0)

from scipy import stats
from mlxtend.preprocessing import minmax_scaling

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
# report on the percentage of missing data in a given data frame
def null_values_report(df):
    missing_values_count = df.isnull().sum()
    total_cells = np.prod(df.shape)
    total_missing = missing_values_count.sum()
    percent_missing = (total_missing/total_cells) * 100
    print(f"Total number of cells: {total_cells}")
    print(f"Total number of empty cells: {total_missing}")
    print(f"Precentage of missing cells: {percent_missing}")

#Gets the mean absolute error(mae) using Decision tree regresor
def get_mae_DTR(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

#Gets the mean absolute error(mae) using Random forest regresor
def get_mae_RF(max_leaf_nodes, train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(train_X, train_y)
    preds_val = model.predict(val_X)
    mae = mean_absolute_error(val_y, preds_val)
    return(mae)

#Gets the least mae in a range of values from 10 - 1000 
def get_least_mae(get_mae):
    lst = [i for i in range(10,1000,10)]
    mae_lst = []
    for i in lst:
        my_mae = get_mae(i, train_X, val_X, train_y, val_y)
        mae_lst.append([i, my_mae])
    mae_lst = np.array(mae_lst)
    mae_lst = mae_lst[mae_lst[:,1].argsort()]
    return mae_lst[0]

In [3]:
train_data = pd.read_csv("train.csv", parse_dates=True)
test_data = pd.read_csv("test.csv", parse_dates=True)
test_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


In [4]:
null_values_report(train_data)

Total number of cells: 118260
Total number of empty cells: 6965
Precentage of missing cells: 5.889565364451209


In [5]:
null_values_report(test_data)

Total number of cells: 116720
Total number of empty cells: 7000
Precentage of missing cells: 5.9972583961617545


In [6]:
train_data_cols = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SalePrice']
test_data_cols = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
hse_train_data = train_data[train_data_cols]
hse_test_data = test_data[test_data_cols]

In [7]:
hse_train_data = hse_train_data.fillna(0)
hse_test_data = hse_test_data.fillna(0)
null_values_report(hse_train_data)
null_values_report(hse_test_data)


Total number of cells: 39420
Total number of empty cells: 0
Precentage of missing cells: 0.0
Total number of cells: 37934
Total number of empty cells: 0
Precentage of missing cells: 0.0


In [8]:
features = ['LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal']
y = hse_train_data['SalePrice']
X = hse_train_data[features]

In [9]:
# for get_mae_DTR or get_mae_RF to work, these variable names must be constant
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)

In [10]:
print(get_least_mae(get_mae_DTR))

[  130.         26629.44533168]


In [11]:
print(get_least_mae(get_mae_RF))

[  320.         20938.07916234]


In [ ]:
#hse_test_data.head()

In [12]:
hse_model = RandomForestRegressor(max_leaf_nodes=320, random_state=1)
hse_model.fit(X, y)
price_predictions = hse_model.predict(hse_test_data)

In [13]:
price_predictions = pd.DataFrame(price_predictions)
price_predictions

,0
0,117953.148359
1,159997.259491
2,181758.898936
3,179456.762591
4,187574.032531
...,...
1454,73064.143212
1455,89903.087272
1456,173121.002339
1457,101050.234543


In [15]:
price_predictions.rename(columns={0:'SalePrice'}, inplace=True)
prices = price_predictions.SalePrice

In [16]:
prices

0       117953.148359
1       159997.259491
2       181758.898936
3       179456.762591
4       187574.032531
            ...      
1454     73064.143212
1455     89903.087272
1456    173121.002339
1457    101050.234543
1458    275867.456432
Name: SalePrice, Length: 1459, dtype: float64

In [18]:
test_data['SalePrice'] = prices
test_data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,117953.148359
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,159997.259491
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,181758.898936
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,179456.762591
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,0,NaN,NaN,NaN,0,1,2010,WD,Normal,187574.032531
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,73064.143212
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,89903.087272
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,173121.002339
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,101050.234543


In [19]:
new_df = test_data[['Id','SalePrice']]

In [20]:
new_df.head()

,Id,SalePrice
0,1461,117953.148359
1,1462,159997.259491
2,1463,181758.898936
3,1464,179456.762591
4,1465,187574.032531


In [21]:
new_df.to_csv("hse_price_predictions.csv")

In [22]:
final_data = pd.read_csv("hse_price_predictions.csv", index_col='Id')

In [23]:
final_data.head()

,Unnamed: 0,SalePrice
Id,,
1461,0,117953.148359
1462,1,159997.259491
1463,2,181758.898936
1464,3,179456.762591
1465,4,187574.032531


In [24]:
data = final_data.drop("Unnamed: 0", axis=1)

In [25]:
data.head()

,SalePrice
Id,
1461,117953.148359
1462,159997.259491
1463,181758.898936
1464,179456.762591
1465,187574.032531


In [26]:
data.to_csv("house_price_predictions.csv")